In [1]:
# TODOs:
# demos (bottom)

# THINGS TO CONSIDER:
# - n_components

# REFERENCES: https://www.slynyrd.com/blog/2018/1/10/pixelblog-1-color-palette
# HEURISTICS FOR HSL: http://hslpicker.com/

In [2]:
import sys
!{sys.executable} -m pip install colr

In [ ]:
from colormath.color_objects import LabColor, XYZColor, sRGBColor, HSLColor, AdobeRGBColor
from colormath.color_conversions import convert_color
from colr import Colr as C
from copy import copy, deepcopy

import itertools

# import matplotlib.pyplot as plt
# import matplotlib as mpl# from matplotlib.patches import Ellipse

class color_library:
    """ Class that represents a color library. The color palette has access to this. """

    def rgb_to_hsl(self, a, b, c):
        rgb = sRGBColor(a, b, c, is_upscaled=True)
        hsl = convert_color(rgb, HSLColor)
        return hsl.get_value_tuple()

    def hsl_to_rgb(self, a, b, c):
        hsl = HSLColor(a, b, c)
        rgb = convert_color(hsl, sRGBColor).get_value_tuple()
        if self.is_valid_rgb(rgb):
            return [rgb[0]*255, rgb[1]*255, rgb[2]*255]
#         return self.correct_rgb(rgb)
        return self.correct_rgb([rgb[0]*255, rgb[1]*255, rgb[2]*255])
        
    def is_valid_rgb(self, color):
        for param in color:
            if param < 0 or param > 255:
                return False;
        return True;
    
    def correct_rgb(self, color):
        color = np.asarray(color)
        for i in range(len(color)):
            if color[i] < 0:
                color[i] = 0
            if color[i] > 255:
                color[i] = 255
        return color
    
    # given an RGB array [r,g,b], returns a string (r, g, b)
    def rgb_to_string(self, rgb):
        rgb_string = "("
        for c in rgb:
            rgb_string += str(c) + ", "
            
        rgb_string = rgb_string[: len(rgb_string) - 2]
        return rgb_string + ")"

    def arr_to_int(self, arr):
        for r in range(len(arr)):
            arr[r] = int(arr[r])
        return arr

    # rgb inputs
    def print_combo(self, fg, bg):
        for i in range(0,3):
            if fg[i] > 255:
                fg[i] = 255
            if bg[i] > 255:
                bg[i]= 255
        print(C().b_rgb(bg[0], bg[1], bg[2]) .rgb(fg[0], fg[1], fg[2], 'Lorem ipsum.'))

    def bound(self, min_val, max_val, val):
        new_val = val
        if (val > max_val):
             new_val = max_val
        elif (val < min_val):
            new_val = min_val
        return new_val
    
    def rgb_to_hex(self, r,g,b):
        return '#%02x%02x%02x' % (int(r), int(g), int(b))
    
    def color_descriptor(self, hue, saturation, lightness):
        if saturation < 5:
            return "grey"
        if lightness < 5:
            return "black"
        elif lightness > 97:
            return "white"
        elif hue <= 10 or hue >= 350:
            return "red"
        elif hue < 40:
            return "orange"
        elif hue < 60:
            return "yellow"
        elif hue < 160:
            return "green"
        elif hue < 250:
            return "blue"
        elif hue < 290:
            return "purple"
        elif hue < 350:
            return "pink"
    
    def color_fun(self, hue, saturation, lightness):
        if saturation < 5:
            return "fog"
        if lightness < 5:
            return "night"
        elif lightness == 98:
            return "snow"
        elif hue <= 10 or hue >= 350:
            return "strawberry"
        elif hue < 40:
            return "mango"
        elif hue < 60:
            return "lemon"
        elif hue < 160:
            return "lime"
        elif hue < 250:
            return "blueberry"
        elif hue < 290:
            return "lilac"
        elif hue < 350:
            return "rose"
        


In [ ]:
import math
import random as r
import numpy as np
from sklearn.mixture import GaussianMixture
from scipy import linalg

class palette_generator:
    """ Class that represents the palette generator model """

    def __init__(self):
        self.color_library = color_library()
        self.color_gmm = GaussianMixture(n_components=3)
        self.hue_gmm = GaussianMixture(n_components=2)
        self.num_steps = 8
        self.num_palettes = 0
        self.randomness = .3 # given the color passed into the stepping wheel, we should diverge from
                             # that color by a certain amount in the beginning to get diverse outputs
                             # this number should lower as we get more samples
                             # 0: never diverges
                             # 1: always change the color
        self.hue_shift_max = 20
        # an array of hue shift index, which measures how much hue shift is desired (*self.hue_shift_max )
        # index = 0: never shifts (0)
        # index = 1: always shifts (20)
        
    # Helper method for testing
    # input: array of hsl samples directly taken from the gmm and converts them
    # into rgb and prints them!
    def print_samples(self, samples):
#         print("----gmm samples----")
        for color in samples:
            rgb = self.color_library.hsl_to_rgb(color[0], color[1], color[2])
            self.color_library.print_combo(rgb,rgb)
            
    # method that takes in rgb colors and palettes that the user likes, and updates
    # hyperparemters such as randomness, num_inputs, and hue preference;
    # this should be called before the stepping wheel, in generate_palettes
    # a palette is a tuple of rgb values like [[r, g, b], [r, g, b]]
    # output: a list of hsl liked colors to be passed into the gmm
    def process_input(self, samples, palettes, if_print):
        hsl_input = []
        
        if palettes == None:
            palettes = []
#         print("given samples:")
        for color in samples:
            if if_print: 
                self.color_library.print_combo(color, color)
            hsl_input.append(self.color_library.rgb_to_hsl(color[0], color[1], color[2]))
        # randomness: decreases randomness for every color we like
        
        self.num_palettes = len(palettes)
        num_inputs = len(samples) + 2 * len(palettes) # assuming each palette has 2 colors
        self.randomness = 1/(0.04 * num_inputs + 1) # over 25 input colors: < 0.5
        
        # hue shift (-|x|) + hue_gmm
#         print("given palettes:")
        hue_shifts = []
        for palette in palettes:
            if if_print:
                self.color_library.print_combo(palette[0], palette[1])
                self.color_library.print_combo(palette[1], palette[0])
            color1 = self.color_library.rgb_to_hsl(palette[0][0], palette[0][1], palette[0][2])
            color2 = self.color_library.rgb_to_hsl(palette[1][0], palette[1][1], palette[1][2])
            hsl_input.append(color1)
            hsl_input.append(color2)
#             print(color1[0])
#             print(color2[0])
#             print(abs(color1[0] - color2[0]))
            hue_diff = math.radians(abs(color1[0] - color2[0]))
#             print("hue_diff for the palette: ", hue_diff)
#             hue_shift = (0.5 * math.sin(hue_diff - 0.5 * math.pi)) + 0.5 # 0/360 -> 0 / 180 -> 1
            hue_shift = 1 - abs(1/math.pi*(hue_diff - math.pi))
#             print("hue_shift for the palette: ", hue_shift)
            hue_shifts.append(hue_shift)
#         print("---")
        if self.num_palettes >= 2:    
            hue_shifts = np.reshape(hue_shifts, (-1, 1))
            self.hue_gmm.fit(hue_shifts) 
#             print("gmm means: ", self.hue_gmm.means_)
#         print("num_inputs:", num_inputs)
#         print("randomness:", self.randomness)
        return hsl_input
    
    def generate_hue_shift(self):
        if self.num_palettes < 2:
            hue_shift = 0.5
        else:
            hs = np.clip(self.hue_gmm.sample(1)[0][0][0], 0, 1)
#             print("hs index sampled from gmm: ", hs)
            if hs > 0.4 and hs < 0.6: # medium shift
                if r.random() < 0.3:
                    hue_shift = abs(r.uniform(0, 0.4) * self.hue_shift_max)
                elif r.random() > 0.7:
                    hue_shift = abs(r.uniform(0.6, 1) * self.hue_shift_max)
                else: # 40% of the time
                    hue_shift = abs(r.uniform(0.4, 0.6) * self.hue_shift_max)
            else:
                if r.random() < hs:
                    hue_shift = abs(r.uniform(hs, 1) * self.hue_shift_max)
                else:
                    hue_shift = abs(r.uniform(0, hs) * self.hue_shift_max)
#         print("hue_shift", hue_shift)
        return hue_shift
        # 0.1: 10% (0.1, 1.0) | 90% (0, 0.1)
    
    # enabling other hues to be generated
    def randomnize_given_color(self, color):
        new_color = deepcopy(color)
        if r.random() < self.randomness:
            new_hue = np.mod(color[0] + r.uniform(30, 100), 360)
            color[0] = new_hue
            new_saturation = self.saturation_clip(color[1] + r.uniform(0, 0.2))
            color[1] = new_saturation
        return color
    
    def saturation_clip(self, value):
        value = abs(value)
        direction = (-1)**int(value)
        return np.mod((1 + direction*np.mod(value, 1)), 1)
    
    def stepping_wheel(self, color):
        self.num_steps= round(r.uniform(6, 8))
        
#         print("given color: " + str(color))
        rgb = self.color_library.hsl_to_rgb(color[0], color[1], color[2])
#         self.color_library.print_combo(rgb, rgb)
        # randomnize colors based on self.randomness to get more diverse outputs
        color = self.randomnize_given_color(color)
        rgb = self.color_library.hsl_to_rgb(color[0], color[1], color[2])
#         print("randomnized color:", color)
#         self.color_library.print_combo(rgb, rgb)

        # center_i: middle of the palette
        center_i = round(self.num_steps/2 + 0.1)-1
            
        # hue: linear increase
        palette_h = np.zeros(self.num_steps)
        hue_shift = self.generate_hue_shift()
#         print("hue shift generated: ", hue_shift)
        for i in range(0, self.num_steps):
            value = color[0] + (i - center_i) * hue_shift
            palette_h[i] = np.mod(value, 360)
        
        # saturation: proportional decrease
        palette_s = np.zeros(self.num_steps)         
        saturation_shift = 0.2
        palette_s[center_i] = np.clip(color[1], 0, 1)
        sign = 1
        for i in range(center_i - 1, -1, -1): # left
            last = palette_s[i + 1] # saturation of last element
            prop = 1.0/(1 + np.exp(-abs(i - center_i)/5.0))  # sigmoid
            value = sign*last - saturation_shift * prop
            sign = value / abs(value)
            palette_s[i] = self.saturation_clip(value)
        sign = 1
        for i in range(center_i + 1, self.num_steps, 1): # right   
            last = palette_s[i - 1] # saturation of last element
            prop = 1.0/(1 + np.exp(-abs(i - center_i)/5.0)) 
            value = sign*last - saturation_shift * prop
            sign = value / abs(value)
            palette_s[i] = self.saturation_clip(value)
        palette_s = np.clip(palette_s, 0.01, 0.99)
        
        # brightness: a*log(x)+1, a in [0.3, 1] (increasing)
        palette_b = np.zeros(self.num_steps)
        # determines the placement of the sampled color in the palette
        center_i = round(self.num_steps/(1.0 + np.exp(3-5*color[2]))) # modified sigmoid
        center_i = np.clip(int(center_i), 0, self.num_steps-1)
#         print("center_i: ", center_i)
        palette_b[center_i] = np.clip(color[2], 0.01, 0.95) # given middle color
        delta_x = 0.05
        # setting a in the a*log(x) + 1
        if (color[2] > 0.49 and color[2] < 0.51): # heuristics for setting the rate of change of brightness
            a = 1.585
        else:
            a = abs(0.1/(color[2]-0.5))**0.2 
        for i in range(center_i - 1, -1, -1): # left
            last = palette_b[i + 1] # brightness of last element
            palette_b[i] = np.clip((last - delta_x * (a/last)**0.5), 0.01, 0.95) # derivative of a*log(x)+1  
        for i in range(center_i + 1, self.num_steps, 1): # right   
            last = palette_b[i - 1] # brightness of last element
            palette_b[i] = np.clip((last + delta_x * (a/last)**0.5), 0.01, 0.95) # derivative of a*log(x)+1
        palette_b = np.clip(palette_b, 0.01, 0.95)
        
        # combining h, s, b
        palette = []
#         print("palette:")
        for i in range(0, self.num_steps):
            c = [palette_h[i], palette_s[i], palette_b[i]]
#             print(c)
            palette.append(c) 
            rgb = self.color_library.hsl_to_rgb(c[0], c[1], c[2])
#             self.color_library.print_combo(rgb, rgb)

        return palette
    
    
    # Given a list of inputs [colors] that are in hsl form [[h,s,l], [h,s,l], [h,s,l] ...]
    # Outputs colors in HSL: [[h,s,l], [h,s,l], [h,s,l]...]
    def sample_gmm(self, hsl_input, num_samples):
        hsl_input = np.reshape(hsl_input, (-1, 3)) # 3 columns
        self.color_gmm.fit(hsl_input)
#         print("color_gmm means: ",self.color_gmm.means_)
        return self.color_gmm.sample(num_samples)[0]
    
    
    # takes in liked colors from quiz and liked palettes?
    # return palettes in form [[[h,s,l], [h,s,l]...], [[h,s,l], [h,s,l] ...]].
    def generate_palettes(self, liked_colors, liked_palettes, num_palettes, if_print):
        hsl_input = self.process_input(liked_colors, liked_palettes, if_print)
        gmm_samples = self.sample_gmm(hsl_input, num_palettes)
        palettes = []
        for color in gmm_samples: # make a palette for each sample from the gmm
            p = self.stepping_wheel(color)
            palettes.append(p)
            rgb1 = self.color_library.hsl_to_rgb(p[0][0], p[0][1], p[0][2])
            rgb2 = self.color_library.hsl_to_rgb(p[self.num_steps - 1][0], p[self.num_steps - 1][1], p[self.num_steps - 1][2])
            if if_print:
                self.color_library.print_combo(rgb1, rgb2)
                self.color_library.print_combo(rgb2, rgb1)
#             print(" -- ")
        return palettes
    
    #

    
    # helper function that translates generated palettes into a json file that is the gallery cards
    def output_to_gallery_cards(self, palettes):
        outputs = []
        
        with open('font_data.json') as f:
            font_data = json.load(f)
            
        font_names = list(font_data.keys())
        
        for palette in palettes:
            color_id = []
            p = {}
            p["colors"] = {}
            p["colors"]["hex"] = []
            p["colors"]["rgb"] = []
            p["colors"]["labels"] = []
            p["colors"]["fun"] = []
            p["title"] = {}
            p["body"] = {}
            
            two_tone = [ palette[0], palette[len(palette) - 1]]
            for color in two_tone:
                rgb = self.color_library.hsl_to_rgb(color[0], color[1], color[2])
                rgb_string =  self.color_library.rgb_to_string(self.color_library.arr_to_int(rgb))
                
                hex_color = self.color_library.rgb_to_hex(rgb[0], rgb[1], rgb[2])
                label = self.color_library.color_descriptor(color[0], color[1], color[2])
                fun_label = self.color_library.color_fun(color[0], color[1], color[2])
                
                # adds all the color data
                p["colors"]["hex"].append(hex_color)
                p["colors"]["rgb"].append(rgb_string)
                p["colors"]["labels"].append(label)
                p["colors"]["fun"].append(fun_label)
                
            # adds title and body information
            text_color = 1 if r.random() > .5 else 0
            bg_color = 0 if text_color == 1 else 1
             
            title_font = r.choice(font_names)
            p["title"]["font"] = title_font
            p["title"]["type"] =  font_data[title_font]["type"]
            p["title"]["color"] = text_color
            
            body_font = r.choice(font_names)
            p["body"]["font"] = body_font
            p["body"]["type"] =  font_data[body_font]["type"]
            p["body"]["color"] = text_color
            
            p["background"] = bg_color
             
                
                
            outputs.append(p)
        
        return outputs
    
        

In [ ]:
# a list of colors we like in rgb, pastels
import json


small_pastel = [ [255, 228, 171], [255, 171, 209], [144, 240, 155], [245, 118, 130], [250, 178, 162], [145, 255, 187], [203, 240, 168]]
small_earth = [ [192, 87, 70], [240, 207, 101], [73, 67, 49], [89, 152, 197], [222, 185, 134], [208, 205, 148], [247, 208, 138]]
small_wack = [ [246,71,64], [248,221,164], [191, 219, 247], [60, 187, 177], [87, 226, 229], [241, 113, 5], [106, 16, 242]]

dark = [ [92, 1, 32] ,[72, 4, 4] ,[77, 10, 24] ,[98, 81, 25] ,[1, 55, 26] ,[1, 54, 28] ,[1, 29, 19] ,[1, 39, 49] ,[2, 45, 21] ,[2, 64, 44] ,[9, 35, 15] ,[4, 19, 34] ,[41, 12, 94] ,[38, 3, 104] ,[0, 7, 65] ,[0, 49, 83] ,[1, 13, 26] ,[1, 22, 53] ,[19, 38, 77] ,[22, 42, 64] ,[27, 2, 69] ,[27, 27, 27] ,[13, 3, 50] ,[65, 74, 76] ]
autumn = [ [225, 104, 101] ,[226, 114, 91] ,[222, 99, 96] ,[208, 95, 4] ,[167, 37, 37] ,[179, 45, 41] ,[180, 51, 50] ,[141, 63, 63] ,[255, 231, 114] ,[255, 239, 161] ,[223, 190, 111] ,[225, 188, 100] ,[96, 73, 19] ,[195, 153, 83] ,[202, 187, 72] ,[207, 181, 59] ,[185, 141, 40] ,[186, 127, 3] ,[220, 178, 12] ,[58, 106, 71] ,[57, 100, 19] ,[95, 167, 119] ,[88, 113, 86] ,[93, 94, 55] ,[168, 152, 155] ,[168, 101, 21] ,[171, 145, 122] ,[173, 120, 27] ,[169, 164, 145] ,[175, 89, 62] ,[136, 83, 66] ,[134, 148, 159] ,[138, 51, 36] ,[135, 124, 123] ,[245, 237, 239] ,[255, 228, 205] ,[77, 61, 20] ]
neon = [ [255, 73, 108] ,[255, 83, 73] ,[255, 43, 43] ,[218, 38, 71] ,[255, 64, 64] ,[205, 74, 76] ,[255, 36, 0] ,[254, 254, 34] ,[253, 255, 0] ,[255, 244, 79] ,[255, 219, 0] ,[69, 206, 162] ,[0, 166, 147] ,[197, 227, 132] ,[0, 127, 255] ,[0, 255, 255] ,[29, 172, 214] ,[31, 194, 194] ,[156, 81, 182] ,[223, 115, 255] ,[219, 145, 239] ,[139, 0, 255] ]
underwater = [ [244, 196, 48] ,[240, 213, 45] ,[236, 242, 69] ,[225, 104, 101] ,[222, 99, 96] ,[226, 114, 91] ,[27, 101, 157] ,[25, 89, 168] ,[20, 80, 170] ,[15, 45, 158] ,[8, 232, 222] ,[24, 167, 181] ,[3, 106, 110] ,[0, 204, 204] ,[0, 255, 255] ,[28, 169, 201] ,[0, 51, 153] ,[0, 86, 167] ,[0, 35, 135] ,[31, 117, 254] ,[25, 116, 210] ,[0, 47, 167] ,[37, 150, 209] ,[32, 46, 84] ,[37, 31, 79] ,[65, 105, 225] ,[59, 145, 180] ]

# concern: the gmm samples black from the neon samples sometime?

red_palettes =  [[ [255, 45, 80] ,[50, 1, 1] ] , [[255, 33, 72] ,[255, 68, 14] ], [[68, 1, 45] ,[225, 104, 101]]]

monochrome =  [[ [250, 224, 181] ,[224, 159, 62] ] , [[158, 42, 43] ,[84, 11, 14] ], [[65, 54, 32] ,[159, 120, 51]]]

very_different = [
[[154, 210, 203],[254, 255, 190]],
[[173, 245, 255], [71 , 40, 54]],
[[233, 114, 76], [189, 198, 150]],
[ [20, 49, 9], [245, 233, 226]],
[[137, 189, 158],[240, 201, 135]]]

mixed_palettes=[
[[194, 231, 217],[38, 63, 139]],
[[252, 109, 171],[247, 246, 197]],
[[152, 210, 235],[178, 177, 207]],
[[255, 192, 159],[252, 245, 199]],
[[281, 224, 242],[82, 21, 78]],
[[18, 69, 89],[174, 195, 176]]
]


generator = palette_generator()
palettes = generator.generate_palettes(underwater, None, 10, False)
data =  generator.output_to_gallery_cards(palettes)
print(data)
# dumps data into json after
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)
    
